In [50]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import random
import array
from numpy import array

# Set hyperparameters
learning_rate = .08
training_epochs = 100
display_steps = 10

# Network parameters
n_hidden = 100 # number of hidden nodes
n_output = 1 # number of unique outputs

n_input = 100
file_len = 4000; 
ninetyperc = int(file_len * .9)


In [51]:
def createInput():
    lineArray=[]
    with open("big.txt") as f:
        for line in f:
            lineArray.append(line.split())
    return lineArray

def createX(data):
    newAr = [i[4:] for i in data]
    return newAr
def createY(data):
    newAr = [i[0] for i in data]
    return newAr



data = createInput()
random.shuffle(data)

train_data = data[:ninetyperc]
test_data = data[ninetyperc:]


x_train = createX(train_data)
x_test = createX(test_data)
y_train = createY(train_data)
y_test = createY(test_data)

x_train = array(x_train) 
x_train = x_train.reshape(x_train.shape[0], n_input)
x_train.shape

x_test = array(x_test)
x_test = x_test.reshape(x_test.shape[0], n_input)

y_train = array(y_train)
y_train = y_train.reshape(y_train.shape[0], n_output)


y_test = array(y_test)
y_test = y_test.reshape(y_test.shape[0], n_output)

y_train.shape


(3600, 1)

In [53]:
# Graph nodes - TensorFlow
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])


# Weights and biases
weights = {
    "hidden" : tf.Variable(tf.random_normal([n_input, n_hidden]), name="weight_hidden"),
    "hidden2" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden2"),
    "hidden3" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden3"),
    "hidden4" : tf.Variable(tf.random_normal([n_hidden, n_hidden]), name="weight_hidden4"),
    "output" : tf.Variable(tf.random_normal([n_hidden, n_output]), name="weight_output")
}

bias = {
    "hidden" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden"),
    "hidden2" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden2"),
    "hidden3" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden3"),
    "hidden4" : tf.Variable(tf.random_normal([n_hidden]), name="bias_hidden4"),
    "output" : tf.Variable(tf.random_normal([n_output]), name="bias_output")
}    
    
# Define the multilayer perceptron (MLP) model
def model(X, weights, bias):
    layer_1 = tf.add(tf.matmul(X, weights["hidden"]), bias["hidden"])
    layer_1 = tf.nn.tanh(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights["hidden2"]), bias["hidden2"])
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2, weights["hidden3"]), bias["hidden3"])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights["hidden4"]), bias["hidden4"])
    layer_4 = tf.nn.tanh(layer_4)
    
    output_layer = tf.matmul(layer_4, weights["output"]) + bias["output"]
    return output_layer


# Define model
pred = model(X, weights, bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.losses.mean_squared_error(Y,pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# Initializing global variables
init = tf.global_variables_initializer()
start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        _, c = sess.run([optimizer, cost], feed_dict={X: x_train, Y: y_train})
        if(epoch + 1) % display_steps == 0:
            print("Epoch: " , (epoch+1), " Cost: ", c)
    print("Optimization finished!")
    
    
    train_result = sess.run(pred, feed_dict={X: x_train})
    test_result = sess.run(pred, feed_dict={X: x_test})
    
    y_test = np.array(y_test,dtype=np.float32) #Converts y_test to float
    

end_time = time.time()

print("Completed in ", end_time - start_time, " seconds.")

count = 0
errorTotal = 0
total = 0
for val in test_result:
    #print("Estimated: ", test_result[count], "Actual: ", y_test[count], "Abs Dif: ", abs(test_result[count]-y_test[count]))
    errorTotal = errorTotal + abs(test_result[count]-y_test[count])
    count = count + 1
    
print("Total Error: ", errorTotal, " Over ", count , " points")
print("Average error: ", errorTotal/count)

count = 0
for val in test_result:
    print("Estimated: ", test_result[count], "Actual: ", y_test[count], "Abs Dif: ", abs(test_result[count]-y_test[count]))
    count = count + 1
    



Epoch:  10  Cost:  18.770638
Epoch:  20  Cost:  6.5719304
Epoch:  30  Cost:  1.6231207
Epoch:  40  Cost:  0.7654674
Epoch:  50  Cost:  0.5787389
Epoch:  60  Cost:  0.53840894
Epoch:  70  Cost:  0.4715678
Epoch:  80  Cost:  0.47139812
Epoch:  90  Cost:  0.45685697
Epoch:  100  Cost:  0.45420843
Optimization finished!
Completed in  12.863476991653442  seconds.
Total Error:  [283.6545]  Over  500  points
Average error:  [0.567309]
Estimated:  [2.883878] Actual:  [2.3830411] Abs Dif:  [0.50083685]
Estimated:  [3.312885] Actual:  [3.882598] Abs Dif:  [0.5697129]
Estimated:  [3.2849834] Actual:  [3.7897675] Abs Dif:  [0.5047841]
Estimated:  [2.8556397] Actual:  [2.322521] Abs Dif:  [0.5331187]
Estimated:  [3.625443] Actual:  [2.8411572] Abs Dif:  [0.7842858]
Estimated:  [3.312885] Actual:  [3.493172] Abs Dif:  [0.18028688]
Estimated:  [3.4418848] Actual:  [3.331622] Abs Dif:  [0.11026287]
Estimated:  [2.3661437] Actual:  [1.6624616] Abs Dif:  [0.70368207]
Estimated:  [3.44707] Actual:  [2.95

Estimated:  [2.3661437] Actual:  [1.835174] Abs Dif:  [0.53096974]
Estimated:  [2.6517265] Actual:  [2.9997222] Abs Dif:  [0.34799576]
Estimated:  [2.8241446] Actual:  [3.7068322] Abs Dif:  [0.88268757]
Estimated:  [2.843571] Actual:  [3.3719723] Abs Dif:  [0.5284014]
Estimated:  [2.885102] Actual:  [2.3382833] Abs Dif:  [0.54681873]
Estimated:  [2.5445871] Actual:  [2.4029567] Abs Dif:  [0.14163041]
Estimated:  [2.760506] Actual:  [3.9406786] Abs Dif:  [1.1801727]
Estimated:  [3.435819] Actual:  [3.4249744] Abs Dif:  [0.01084447]
Estimated:  [2.5445797] Actual:  [3.3965495] Abs Dif:  [0.8519697]
Estimated:  [2.8097804] Actual:  [3.2398424] Abs Dif:  [0.43006206]
Estimated:  [3.4994552] Actual:  [2.6835036] Abs Dif:  [0.8159516]
Estimated:  [2.0786457] Actual:  [2.225341] Abs Dif:  [0.14669538]
Estimated:  [2.3504527] Actual:  [2.8095725] Abs Dif:  [0.4591198]
Estimated:  [3.3929336] Actual:  [3.57515] Abs Dif:  [0.1822164]
Estimated:  [2.5445802] Actual:  [3.418339] Abs Dif:  [0.87375